# Quiz

In [29]:
from bs4 import BeautifulSoup
from zipfile import ZipFile
import glob # just to check files with .html extension
import os
import pandas as pd

# Gathering (Coleta)

In [40]:
data_folder = 'data'
html_folder = os.path.join(data_folder, 'rt_html')

if (os.path.exists(html_folder) and (len(glob.glob1(html_folder, '*.html')) == 100)):
    print('All files was already extracted...')
else:
    if(os.path.exists(f'{html_folder}.zip')):
#         Extract all files to the destination folder
        with ZipFile(os.path.join(data_folder, 'rt_html.zip'), 'r') as datazip:
            datazip.extractall(html_folder)
    else:
#         Data zip file was not found
        print(f'Data zip file wasn\'t found at {data_folder}')
        

All files was already extracted...


In [81]:
# List of dictionaries to build file by file anda later convert to a DataFrame
df_list = []

for movie_html in os.listdir(html_folder):
    with open(os.path.join(html_folder, movie_html)) as file:
        soup = BeautifulSoup(file, 'lxml')
        
        title = soup.title.contents[0][0:-len(' - Rotten Tomatoes')]
        audience_score = int(soup.select('.audience-score.meter')[0].find('span').contents[0][:-1])
        num_audience_ratings = soup.
        
        
#         df_list.append({
#             'title': soup.title.contents[0],
#             'audience_score': soup.audience_score,
#             'number_of_audience_ratings': int()
#         })
        
        
        print(audience_score)
        

97
86
89
94
89
95
87
86
91
95
91
92
95
92
90
92
89
78
90
94
94
94
90
94
82
89
85
89
94
81
87
87
90
94
82
72
86
87
98
97
93
88
80
89
88
89
93
90
90
94
92
82
91
86
77
87
95
80
90
94
87
95
92
93
72
94
93
87
94
87
86
97
95
86
93
91
89
95
93
72
95
90
89
91
94
97
86
93
89
88
93
92
86
89
93
90
93
95
90
92
